In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('drive/My Drive/dataset/greetings_small.csv')
data = data['greetings'].str.replace('[^\w\s]',' ')
data = data.str.lower()
data

0       тебе желаю море счастья улыбок  солнца и тепла...
1       с днем рождения поздравляюи желаю день за днем...
2       поздравляю с днем рождения  пусть жизнь дарит ...
3       пусть в жизни будет все  что нужно здоровье  м...
4       с днем рожденья поздравляюи от всей души желаю...
                              ...                        
4992    желаю  по жизни шагать лишь вперёд упорством с...
4993    пускай удача не отступит легко поможет в каждо...
4994    желаю искромётного успеха чтоб воплощались все...
4995    желаю громкого везения пускай успехи валят с н...
4996    желаю море процветания во всех задачах   продв...
Name: greetings, Length: 4997, dtype: object

In [0]:

from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Neural Net Layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding

# Neural Net Training
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [0]:
max_words = 50000 # Max size of the dictionary
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data.values)
sequences = tokenizer.texts_to_sequences(data.values)
print(sequences[:5])

[[12, 7, 64, 14, 89, 133, 1, 70, 6, 25, 82, 96, 566, 53, 68, 447, 1340, 3, 2, 111, 11, 38, 43, 287, 188, 1, 271, 80, 189, 77, 73, 305, 547, 168, 28, 128, 1155, 1636, 632, 331, 1, 3, 9, 8723, 17, 694, 36, 882], [5, 15, 16, 369, 7, 8, 68, 8724, 2422, 1, 219, 137, 115, 68, 915, 97, 22, 28, 35, 90, 1, 70, 6, 189, 143, 2051, 119, 84, 55, 3, 945, 11, 98, 1095, 1, 23, 2221, 1519, 448, 158, 172, 209], [33, 5, 15, 16, 3, 25, 276, 12, 98, 116, 458, 1, 183, 10, 85, 1341, 2052, 1, 476, 122, 7, 20, 2, 548, 111, 9, 3642, 34, 1, 1520, 1, 3, 17, 663, 38, 2423, 497, 2053, 80, 1, 527, 349], [3, 2, 21, 11, 10, 30, 382, 78, 114, 46, 1, 489, 4, 5761, 3, 72, 53, 1342, 106, 47, 3, 34, 11, 1897, 39, 549, 1, 31, 8725, 7, 35, 272, 4451, 1755, 822, 1, 161], [5, 15, 27, 369, 26, 125, 63, 5762, 23, 94, 20, 535, 66, 58, 90, 3, 30, 435, 695, 102, 19, 34, 282, 1430, 175, 3643, 220, 3, 4, 2693, 1010, 2, 111, 3, 332, 567, 2, 764, 11, 188, 155, 600, 1155, 96, 285, 33]]


In [0]:
text = [item for sublist in sequences for item in sublist]
vocab_size = len(tokenizer.word_index)
print('Vocabulary size in this corpus: ', vocab_size)

Vocabulary size in this corpus:  23691


In [0]:
sentence_len = 13
pred_len = 1
train_len = sentence_len - pred_len
seq = []
# Sliding window to generate train data
for i in range(len(text)-sentence_len):
    seq.append(text[i:i+sentence_len])
# Reverse dictionary to decode tokenized sequences back to words
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

In [0]:
trainX = []
trainy = []
for i in seq:
    trainX.append(i[:train_len])
    trainy.append(i[-1])

In [0]:
model = Sequential([
    Embedding(vocab_size+1, 50, input_length=train_len),
    LSTM(150, return_sequences=True),
    LSTM(150),
    Dense(150, activation='relu'),
    Dense(vocab_size, activation='softmax')
])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
import numpy as np
from numpy import array


# model.fit(np.asarray(trainX),
#          pd.get_dummies(np.asarray(trainy)),
#          epochs = 500,
#          batch_size = 10240,
#          callbacks = callbacks_list,
#          verbose = 2)

In [0]:

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(np.asarray(trainX), pd.get_dummies(np.asarray(trainy)), batch_size=128, epochs=300)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 210282 samples
Epoch 1/300
210282/210282 [==============================] - 70s 331us/sample - loss: 7.3245 - acc: 0.0512
Epoch 2/300
210282/210282 [==============================] - 67s 317us/sample - loss: 6.9259 - acc: 0.0714
Epoch 3/300
210282/210282 [==============================] - 67s 320us/sample - loss: 6.6137 - acc: 0.0821
Epoch 4/300
210282/210282 [==============================] - 67s 319us/sample - loss: 6.3921 - acc: 0.0895
Epoch 5/300
210282/210282 [==============================] - 67s 317us/sample - loss: 6.1982 - acc: 0.0958
Epoch 6/300
210282/210282 [==============================] - 66s 316us/sample - loss: 6.0167 - acc: 0.1026
Epoch 7/300
210282/210282 [==============================] - 66s 315us/sample - loss: 5.8441 - acc: 0.1068
Epoch 8/300
210282/210282 [==============================] - 67s 317us/sample - loss: 5.6851 - acc: 0.1123
Epoch 9/300
210282/210282 

In [0]:
import pickle
pickle.dump(tokenizer, open('drive/My Drive/dataset/tokenizer.pkl', 'wb'))
model.save('drive/My Drive/dataset/model_weights.hdf5')

In [0]:

pickle.dump(tokenizer, open('tokenizer.pkl', 'wb'))
model.save('model_weights.hdf5')
from google.colab import files
files.download('tokenizer.pkl')
files.download('model_weights.hdf5')


MessageError: ignored

In [0]:
import pickle
pickle.dump(reverse_word_map, open('drive/My Drive/dataset/reverse_word_map.pkl', 'wb'))